In [1]:
# !pip install gurobipy

In [1]:
from gurobipy import *

## 問題例12 CVaRを導入

In [44]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

# 期待値と分散共分散行列の準備
mean = np.array([1, 2, 3, 4])
cov = np.array([[4,1.2,-1.3,0.6],
               [1.2,3,1.5,-2.0],
               [-1.3,1.5,2,0.9],
               [0.6,-2.0,0.9,1.5]])

# numpy を用いた生成
size=10
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((10,4)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

<ipython-input-44-df4ba3a8ff79>:15: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [52]:
# nと初期配置を変更しなければいけない

O1=[1,2,3,4]

S=[i for i in range(1,5)]
H=[i for i in range(1,5)]
P=[i for i in range(1,5)]
T=[i for i in range(1,7)]
n_=[1,2,4,3]
n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O
O

[[1, 2, 3, 4],
 [1, 3, 4, 2],
 [1, 3, 2, 4],
 [2, 1, 4, 3],
 [1, 3, 2, 4],
 [2, 1, 3, 4],
 [1, 4, 2, 3],
 [3, 4, 2, 1],
 [1, 2, 3, 4],
 [1, 4, 2, 3]]

In [46]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z={},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:])-alpha)

In [47]:
# 初期配置の制約

m.addConstr(x[1,1,O1[3],1]==1)
m.addConstr(x[1,2,O1[3],1]==1)
m.addConstr(x[1,3,O1[2],1]==1)
m.addConstr(x[2,1,O1[3],1]==1)
m.addConstr(x[2,2,O1[2],1]==1)
m.addConstr(x[2,3,O1[1],1]==1)
m.addConstr(x[3,1,O1[2],1]==1)
m.addConstr(x[3,2,O1[2],1]==1)
m.addConstr(x[4,1,O1[1],1]==1)
m.addConstr(x[4,2,O1[0],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [48]:
beta=0.8
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(z[s,h,i] for s in S for h in H if h!=1 for i in range(len(O)))/((1-beta)*len(O)))

In [49]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1499 rows, 565 columns and 9206 nonzeros
Model fingerprint: 0xa09ea36b
Variable types: 121 continuous, 444 integer (444 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 523 rows and 183 columns
Presolve time: 0.09s
Presolved: 976 rows, 382 columns, 5911 nonzeros
Variable types: 85 continuous, 297 integer (297 binary)
Found heuristic solution: objective 1.0000300

Root relaxation: objective 2.584810e-05, 685 iterations, 0.07 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00003    0  132    1.00003    0.00003   100%     -    0s
H    0     0                  

In [50]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print(alpha.X)

x[ 1, 1, 4, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 1, 3]=1.0
x[ 1, 3, 3, 1]=1.0
x[ 2, 1, 4, 1]=1.0
x[ 2, 2, 3, 1]=1.0
x[ 2, 3, 2, 1]=1.0
x[ 3, 1, 3, 1]=1.0
x[ 3, 2, 3, 1]=1.0
x[ 3, 3, 3, 2]=1.0
x[ 3, 4, 3, 5]=1.0
x[ 4, 1, 2, 1]=1.0
x[ 4, 2, 1, 1]=1.0
x[ 4, 2, 2, 4]=1.0
y[ 1, 3, 1]=1.0
y[ 2, 3, 4]=1.0
y[ 2, 4, 3]=1.0
y[ 4, 1, 2]=1.0
z[ 1, 3, 7]=1.0
the objective function 0.50004
2.7755575615628914e-17


In [53]:
O[7]

[3, 4, 2, 1]

In [42]:
O

[[1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [3, 1, 2, 4],
 [3, 4, 1, 2],
 [3, 4, 1, 2],
 [3, 4, 1, 2],
 [1, 2, 4, 3],
 [1, 2, 4, 3],
 [1, 2, 4, 3],
 [2, 3, 1, 4]]